# Pseudo-compressible Navier-Stokes element symbolic formulation and implementation
**Rubén Zorrilla, Riccardo Rossi**
UPC BarcelonaTech, CIMNE

The weakly-compressible Navier-Stokes problem is conformed by the linear momentum conservation equation
\begin{equation*}
\frac{\partial\rho\mathbf{u}}{\partial t} + \rho\mathbf{a}\cdot\nabla\mathbf{u} - \nabla\cdot\mathbf{\boldsymbol{\sigma}} + \frac{\mathbf{u}}{\kappa} = \rho\mathbf{f}
\end{equation*}
together with the mass conservation equation
\begin{equation*}
\frac{D\rho}{Dt} + \rho\nabla\cdot\mathbf{u} = 0
\end{equation*}
with $\mathbf{u}$ denoting the velocity field, $\rho$ the density and $\mathbf{f}$ the volumetric force.
For the sake of generality a Darcy term governed by the $\kappa$ the permeability is also included in the formulation.

Developing the material time derivative of the mass conservation equation one obtains the equation
\begin{equation*}
\frac{\partial\rho}{\partial t} + \mathbf{u}\cdot\nabla\rho + \rho\nabla\cdot\mathbf{u} = 0\; ,
\end{equation*}
which intentionally keeps the $\nabla \rho$ term as it might be non-negligible in the weakly-compressible scenario.

The density local time derivative can be rewritten in terms of the pressure time derivative as
\begin{equation*}
\frac{\partial p}{\partial t} = \frac{\partial p}{\partial\rho}\frac{\partial\rho}{\partial t} \longrightarrow \frac{\partial\rho}{\partial t} = \left(\frac{\partial p}{\partial\rho}\right)^{-1}\frac{\partial p}{\partial t}\; .
\end{equation*}

Introducing the state equation
\begin{equation*}
\frac{\partial p}{\partial\rho} = c^{2}
\end{equation*}
into the mass conservation equation yields
\begin{equation*}
\frac{1}{c^{2}}\frac{\partial p}{\partial t} + \mathbf{u}\cdot\nabla\rho + \rho\nabla\cdot\mathbf{u} = 0
\end{equation*}
where $c$ is the sound velocity of the considered fluid.

Once arrived to this point, one can divide the previous equation by $\rho$ for the sake of having a better conditioned matrix (specially in those cases in where $\rho\gg1$).
This has been implemented as an user-defined option in the symbolic element generation script. Note that this will also affect the stabilization terms.

The density is considered to be time independent in the momentum conservation equation, implying that
\begin{equation*}
\rho\frac{\partial\mathbf{u}}{\partial t} + \rho\mathbf{a}\cdot\nabla\mathbf{u} - \nabla\cdot\mathbf{\boldsymbol{\sigma}} + \frac{\mathbf{u}}{\kappa} = \rho\mathbf{f}\; .
\end{equation*}

In addition, the stress tensor $\boldsymbol{\sigma}$ can be split in an isochoric and deviatoric (viscous) contribution as 
\begin{equation*}
\boldsymbol{\sigma} = -p\mathbf{I} + \rho(\nabla\cdot\mathbf{u})\mathbf{I} + \boldsymbol{\tau}\; .
\end{equation*}
Taking into account that no volumetric deformation is considered in the momentum conservation equation, $\boldsymbol{\sigma}$ can be simplified to
\begin{equation*}
\boldsymbol{\sigma}=-p\mathbf{I}+\boldsymbol{\tau}\; .
\end{equation*}
where $p$ is the pressure and $\boldsymbol{\tau}$ the shear stress tensor, which is obtained from the corresponding constitutive model as
\begin{equation*}
\boldsymbol{\tau} = \mathbb{\mathbb{C}}\nabla^{s}\mathbf{u}\; ,
\end{equation*}
being $\nabla^{s}$ the symmetric gradient operator and $\mathbb{C}$ the 4th order constitutive tensor.

Substituting the stress tensor above, the linear momentum equation reads
\begin{equation*}
\rho\frac{\partial\mathbf{u}}{\partial t} + \rho\mathbf{a}\cdot\nabla\mathbf{u} - \nabla\cdot\left(-p\mathbf{I}+\boldsymbol{\tau}\right) + \frac{\mathbf{u}}{\kappa} = \rho\mathbf{f}
\end{equation*}

Finally, the linear momentum and mass conservation residuals can be defined as
\begin{align*}
\mathbf{R}^{M}(\mathbf{u},p) = & \rho\mathbf{f} - \rho\frac{\partial\mathbf{u}}{\partial t} - \rho\mathbf{a}\cdot\nabla\mathbf{u} + \nabla\cdot\left(-p\mathbf{I}+\boldsymbol{\tau}\right) - \frac{\mathbf{u}}{\kappa} \\
\mathbf{R}^{C}(\mathbf{u},p) = & -\frac{1}{c^{2}}\frac{\partial p}{\partial t} - \mathbf{u}\cdot\nabla\rho - \rho\nabla\cdot\mathbf{u}
\end{align*}

## Galerkin weak form and ASGS stabilization

Considering the velocity test function $\mathbf{w}$ and pressure test function $q$, the Galerkin weak form of the problem can be obtained as

\begin{equation*}
\left(\mathbf{w},\mathbf{R}^{M}(\mathbf{u},p)\right)_{\Omega}+\left(q,\mathbf{R}^{C}(\mathbf{u},p)\right)_{\Omega} = 0\; ,
\end{equation*}
that is
\begin{equation*}
\left(\mathbf{w}, \rho\mathbf{f} - \rho\frac{\partial\mathbf{u}}{\partial t} - \rho\mathbf{a}\cdot\nabla\mathbf{u} + \nabla\cdot\left(-p\mathbf{I}+\boldsymbol{\tau}\right) - \frac{\mathbf{u}}{\kappa}\right)_{\Omega} + \left(q, -\frac{1}{c^{2}}\frac{\partial p}{\partial t} - \mathbf{u}\cdot\nabla\rho - \rho\nabla\cdot\mathbf{u}\right)_{\Omega} = 0\; .
\end{equation*}

On top of that, the following multiscale decomposition is considered

\begin{align*}
\mathbf{u} & = \mathbf{u_{h}}+\mathbf{u_{s}}\\
p & = p_{h}+p_{s}
\end{align*}

where $\mathbf{u_{\mathbf{h}}}$ and $p_{h}$ belong to the finite element space while $\mathbf{u_{s}}$ and $p_{s}$ are the so called unresolvable subscales. Substituting the proposed decomposition in the Galerkin residual weak form above yields

\begin{equation*}
\left(\mathbf{w},\rho\mathbf{f} - \rho\frac{\partial(\mathbf{u}_{h}+\mathbf{u}_{s})}{\partial t} - \rho\mathbf{a}\cdot\nabla(\mathbf{u}_{h}+\mathbf{u}_{s}) + \nabla\cdot\left(-(p_{h}+p_{s})\mathbf{I}+\mathbb{\mathbb{C}}\nabla^s(\mathbf{u}_{h}+\mathbf{u}_{s})\right) - \frac{\left(\mathbf{u}_{h}+\mathbf{u}_{s}\right)}{\kappa} \right)_{\Omega} + \left(q,-\frac{1}{c^{2}}\frac{\partial(p_{h}+p_{s})}{\partial t} - \mathbf{a}\cdot\nabla\rho - \rho\nabla\cdot(\mathbf{u}_{h}+\mathbf{u}_{s})\right)_{\Omega} = 0
\end{equation*}

At this point it is important to clearly state the assumptions concerning the subscales that are to be taken from now on
- Quasi-static subscales: $\frac{\partial\mathbf{u_{s}}}{\partial t}=\frac{\partial p_{s}}{\partial t}\approx0$
- $\mathbf{u_{s}}\approx p_{s}\approx0$ on $\Gamma$
- Linear order finite elements implying that higher order derivatives are equal to 0

Developing terms in the decomposed Galerkin residual weak form considering the quasi-static nature of the subscales one obtains
\begin{align*}
\left(\mathbf{w},\rho\mathbf{f}\right)_{\Omega} &- \left(\mathbf{w},\rho\frac{\partial\mathbf{u}_{h}}{\partial t}\right)_{\Omega} - \left(\mathbf{w},\rho\mathbf{a}\cdot\nabla\mathbf{u}_{h}\right)_{\Omega} - \left(\mathbf{w},\rho\mathbf{a}\cdot\nabla\mathbf{u}_{s}\right)_{\Omega} - \left(\mathbf{w},\nabla p_{h}\right)_{\Omega} - \left(\mathbf{w},\nabla p_{s}\right)_{\Omega} + \left(\mathbf{w},\nabla\cdot\mathbb{\mathbb{C}}\nabla^s\mathbf{u}_{h}\right)_{\Omega} + \left(\mathbf{w},\nabla\cdot\mathbb{\mathbb{C}}\nabla^s\mathbf{u}_{s}\right)_{\Omega} \\ 
&- \left(\mathbf{w},\frac{\mathbf{u}_{h}}{\kappa}\right)_{\Omega} - \left(\mathbf{w},\frac{\mathbf{u}_{s}}{\kappa}\right)_{\Omega} - \left(q,\frac{1}{c^{2}}\frac{\partial p_{h}}{\partial t}\right)_{\Omega} - \left(q,\mathbf{a}\cdot\nabla\rho\right)_{\Omega} - \left(q,\rho\nabla\cdot\mathbf{u}_{h}\right)_{\Omega} - \left(q,\rho\nabla\cdot\mathbf{u}_{s}\right)_{\Omega} = 0 \; .
\end{align*}

Integrating by parts the shear stress terms
\begin{align*}
\left(\mathbf{w},\nabla\cdot\mathbb{\mathbb{C}}\nabla^s\mathbf{u}_{h}\right)_{\Omega} & = -\left(\nabla \mathbf{w},\mathbb{\mathbb{C}}\nabla^s\mathbf{u}_{h}\right)_{\Omega} + \langle\mathbf{w},\left(\mathbb{C}\nabla^s\mathbf{u}_{h}\right)\cdot\mathbf{n}\rangle_{\Gamma}\; ,\\
\left(\mathbf{w},\nabla\cdot\mathbb{\mathbb{C}}\nabla^s\mathbf{u}_{s}\right)_{\Omega} & = -\left(\nabla \mathbf{w},\mathbb{\mathbb{C}}\nabla^s\mathbf{u}_{s}\right)_{\Omega} + \langle\mathbf{w},\left(\mathbb{C}\nabla^s\mathbf{u}_{s}\right)\cdot\mathbf{n}\rangle_{\Gamma} \approx - \left(\nabla\mathbf{w},\mathbb{\mathbb{C}}\nabla^s\mathbf{u}_{s}\right)_{\Omega}\; ,
\end{align*}
and doing so for the pressure ones
\begin{align*}
\left(\mathbf{w},\nabla p_{h}\right)_{\Omega} & = -\left(\nabla\cdot\mathbf{w},p_{h}\right)_{\Omega} + \langle\mathbf{w}\cdot\mathbf{n}, p_{h}\rangle_{\Gamma}\\
\left(\mathbf{w},\nabla p_{s}\right)_{\Omega} & = -\left(\nabla\cdot\mathbf{w},p_{s}\right)_{\Omega} + \langle\mathbf{w}\cdot\mathbf{n}, p_{s}\rangle_{\Gamma} \approx -\left(\nabla\cdot\mathbf{w},p_{s}\right)_{\Omega}\; .
\end{align*}

Complementary, the subscales mass conservation term is also integrated by parts as
\begin{equation*}
\left(q,\rho\nabla\cdot\mathbf{u}_{s}\right)_{\Omega} = -\left(\nabla q,\rho\mathbf{u}_{s}\right)_{\Omega} + \langle\left(\rho q\mathbf{n}\right),\cdot \mathbf{u}_{s}\rangle_{\Gamma} \approx - \left(\nabla q,\rho\mathbf{u}_{s}\right)_{\Omega}\; ,
\end{equation*}
as well as the subscales convective term
\begin{equation*}
\left(\mathbf{w},\rho\mathbf{a}\cdot\nabla\mathbf{u}_{s}\right)_{\Omega} = -\left((\rho(\nabla\cdot\mathbf{a})\mathbf{w} + \rho\mathbf{a}\cdot\nabla\mathbf{w}),\mathbf{u}_{s}\right)_{\Omega} + \langle(\mathbf{n}\cdot(\mathbf{a}\otimes\mathbf{w})), \mathbf{u}_{s}\rangle_{\Gamma} \approx -\left((\rho(\nabla\cdot\mathbf{a})\mathbf{w}+\rho\mathbf{a}\cdot\nabla\mathbf{w}),\mathbf{u}_{s}\right)_{\Omega} \; .
\end{equation*}

The subscale values are obtained from the finite element approximation residuals (Algebraic SubGrid Scales, ASGS) as
\begin{align*}
\mathbf{u_{s}} =& \tau_{1}\mathbf{R}^{M}(\mathbf{u}_{h},p_{h})\\
p_{s} =& \tau_{2}\mathbf{R}^{C}(\mathbf{u}_{h},p_{h})
\end{align*}
being $\tau_{1}$ and $\tau_{2}$ the stabilization coefficients
\begin{align*}
\tau_{1} =& \left(\frac{\rho\tau_{d}}{\Delta t} + \frac{c_{1}\mu}{h^{2}} + \frac{c_{2}\rho||\mathbf{a}||}{h} + \frac{c_{3}}{\kappa h^2}\right)^{-1}\; , \\
\tau_{2} =& \mu + \frac{c_{2}\rho h||\mathbf{a}||}{c_{1}} + \frac{c_{3}}{\kappa c_{1}}\; .
\end{align*}
being $c_{1}=4$, $c_{2}=2$, $c_{3}=2$ and $h$ the characteristic element size.
Note that the selected subscales model contains higher order derivatives that vanish when using linear finite elements.
Also note that the permeability $\kappa$ can be alternatively expressed in terms of the resistance $\sigma = 1 / \kappa$.

Thus, the final Galerkin residual weak form reads
\begin{align*}
\left(\mathbf{w},\rho\mathbf{f}\right)_{\Omega} -& \left(\mathbf{w},\rho\frac{\partial\mathbf{u}_{h}}{\partial t}\right)_{\Omega} - \left(\mathbf{w},\rho\mathbf{a}\cdot\nabla\mathbf{u}_{h}\right)_{\Omega} + \left(\rho(\nabla\cdot\mathbf{a})\mathbf{w},\mathbf{u}_{s}\right)_{\Omega} + \left(\rho\mathbf{a}\cdot\nabla\mathbf{w},\mathbf{u}_{s}\right)_{\Omega} - \left(\nabla\cdot\mathbf{w},p_{h}\right)_{\Omega} - \left(\nabla\cdot\mathbf{w},p_{s}\right)_{\Omega} - \left(\nabla\mathbf{w},\mathbb{\mathbb{C}}\nabla^s\mathbf{u}_{h}\right)_{\Omega} - \left(\nabla\mathbf{w},\mathbb{\mathbb{C}}\nabla^s\mathbf{u}_{s}\right)_{\Omega}\\ 
&- \left(\mathbf{w},\frac{\mathbf{u}_{h}}{\kappa}\right)_{\Omega} - \left(\mathbf{w},\frac{\mathbf{u}_{s}}{\kappa}\right)_{\Omega} - \left(q,\frac{1}{c^{2}}\frac{\partial p_{h}}{\partial t}\right)_{\Omega} - \left(q,\mathbf{a}\cdot\nabla\rho\right)_{\Omega} - \left(q,\rho\nabla\cdot\mathbf{u}_{h}\right)_{\Omega} + \left(\nabla q,\rho\mathbf{u}_{s}\right)_{\Omega} + \langle\mathbf{w}, \left(\mathbb{C}\nabla^s\mathbf{u}_{h}-p_{h}\mathbf{I}\right)\cdot\mathbf{n}\rangle_{\Gamma} = 0 \; .
\end{align*}
Note that the term $\left(\nabla\mathbf{w},\mathbb{\mathbb{C}}\nabla^s\mathbf{u_{s}}\right)_{\Omega}$ vanishes if linear elements are used in the subscale approximation. 

## SYMBOLS TO BE EMPLOYED

Shape functions $N_{I}$ and derivatives $\nabla N_{I}$stored respectively
in a vector $\mathbf{N}$ and a matrix $\mathbf{DN}$. Besides the
following arrays are defined:

- $\mathbf{p}$ such that $p_{I}$ is the current step pressure of node I
- $\mathbf{p_{n}}$ such that $p_{n,I}$ is the previous step pressure of node I
- $\mathbf{p_{nn}}$ such that $p_{nn,I}$ is the two previous step pressure of node I
- $\mathbf{v}$ such that $\mathbf{v}_{IK}$ is the current step velocity of node I, component K
- $\mathbf{v}_{n}$ such that $\mathbf{v}_{n,IK}$ is the previous step velocity of node I, component K
- $\mathbf{v}_{nn}$ such that $\mathbf{v}_{nn,IK}$ is the two previous steps velocity of node I, component K
- $\mathbf{w}$ such that $\mathbf{w}_{I}$ is the linear momentum conservation test function value I at the considered Gauss point
- $\mathbf{q}$ such that $\mathbf{q}_{I}$ is the mass conservation test function value I at the considered Gauss point

Values on the gauss points are expressed in terms of such variables as

- $\mathbf{v\_gauss}:=\mathbf{v}^{T}\mathbf{N}$ a 3x1 matrix
- $\mathbf{p\_gauss}:=\mathbf{p}^{T}\mathbf{N}$ a 1x1 matrix
- $\mathbf{f\_gauss}:=\mathbf{f}^{T}\mathbf{N}$ a 3x1 matrix
- $\mathbf{w\_gauss}:=\mathbf{w}^{T}\mathbf{N}$ a 3x1 matrix
- $\mathbf{q\_gauss}:=\mathbf{q}^{T}\mathbf{N}$ a 1x1 matrix
- $\mathbf{accel\_gauss}:=\left(BDF_{0}\mathbf{V}+BDF_{1}\mathbf{V}_{n}+BDF_{2}\mathbf{V}_{nn}\right)^{T}\mathbf{N}$ a 3x1 matrix
- $\mathbf{pder\_gauss}:=\left(BDF_{0}p+BDF_{1}p_{n}+BDF_{2}p_{nn}\right)\mathbf{N}$ a 1x1 matrix
- $\mathbf{v_{h}:=v^{T}N}$ a 3x1 matrix
- $\mathbf{w_{h}:=w^{T}N}$ a 3x1 matrix
- $p_{h}:=\mathbf{p^{T}N}$ a 1x1 matrix
- $q_{h}:=\mathbf{q^{T}N}$ a 1x1 matrix
- $\mathbf{div\_p}:=\mathbf{\nabla\cdot p}$ a 1x1 matrix
- $\mathbf{div\_v}:=\mathbf{\nabla\cdot v}$ a 1x1 matrix
- $\mathbf{div\_vconv}:=\mathbf{\nabla\cdot vconv}$ a 1x1 matrix
- $\mathbf{grad\_p}:=\mathbf{DN^{T}\cdot p}$ a 3x1 matrix
- $\mathbf{grad\_v}:=\mathbf{DN^{T}\cdot v}$ a 3x3 matrix
- $\mathbf{grad\_vconv}:=\mathbf{DN^{T}\cdot vconv}$ a 3x3 matrix
- $\mathbf{grad\_q}:=\mathbf{DN^{T}\cdot q}$ a 3x1 matrix
- $\mathbf{grad\_w}:=\mathbf{DN^{T}\cdot w}$ a 3x3 matrix
- $\mathbf{grad\_sym\_v}:=$ a 3x3 matrix (symmetric gradient in Voigt form)
- $\mathbf{grad\_sym\_w}:=$ a 3x3 matrix (symmetric gradient of the test function in Voigt form)
- $\mathbf{convective\_term}:=\mathbf{\left(a_{h}\cdot grad\_vh\right)^{T}}$ a 3x1 vector in where $\mathbf{a_{h}}$ is the convective velocity. If the option ''Picard'' linearisation is set, the convective velocity is defined as a symbol $\mathbf{v_{conv}}$ meaning that it is treated
as a constant in the automatic differenctiation. On the contrary, if the option ``FullNR'' linearisation is set, the convective velocity
is defined as $(\mathbf{v_{h}}-\mathbf{v_{mesh}})$ implying that $\mathbf{v_{h}}$ is considered in the automatic differenctiation.
- $\mathbf{stress}:$ 6x1 shear stress vector computed by constitutive law using the previous iteration database. This stress vector is considered
in the computation of the RHS, which is the residual of the previous iteration solution. When computing the LHS (RHS derivative w.r.t.
the DOFs), the $\mathbf{stress}$ is substituted by $\mathbf{C*grad\_sym\_v}$ to consider the stress in the automatic differenctiation.

## IMPLEMENTATION
The residual functional implementation is splitted in two pieces
- $\mathbf{rv\_galerkin}$: functional corresponding to the standard incompressible Navier-Stokes equations. If "artificial\_compressibility" is set as True, the artificial compressibility terms are also added to the functional.
- $\mathbf{rv\_stab}$: functional corresponding to the standard incompressible Navier-Stokes ASGS stabilization. If "artificial\_compressibility" is set as True, the artificial compressibility stabilization terms are also added to the stabilization functional.
- $\mathbf{rv}=\mathbf{rv\_galerkin}+\mathbf{rv\_stab}$: summation of functionals. This is the functional used to compute the LHS and RHS elemental matrices.

For the definition of the subscales, the residuals are splitted in
- $\mathbf{vel\_residual}$: linear momentum conservation equation residual. 
- $\mathbf{mas\_residual}$: mass conservation equation residual. If "artificial\_compressibility" is set as True, the artificial compressibility terms are also added to the residual.


